In [1]:
# Prepare the input for batch submission in polyphen2
# MCT8 UniProtID = P36021
# In Uniprot is the short isoform the one that is represented
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

mct8 = pd.read_csv('MCT8_IA_dataset_T4.csv', index_col=0)

In [2]:
d_aa = {'Cys': 'C', 'Asp': 'D', 'Ser': 'S', 'Gln': 'Q', 'Lys': 'K',
        'Ile': 'I', 'Pro': 'P', 'Thr': 'T', 'Phe': 'F', 'Asn': 'N',
        'Gly': 'G', 'His': 'H', 'Leu': 'L', 'Arg': 'R', 'Trp': 'W',
        'Ala': 'A', 'Val': 'V', 'Glu': 'E', 'Tyr': 'Y', 'Met': 'M',
        'Ter': 'STOP'}
aa_d = {k:v for v,k in d_aa.items()}

----
Generates the Input for **PolyPhen2**

---

In [3]:
mct8['UniProtID'] = 'P36021'
mct8['ShortPosition'] = mct8.LongPosition-74
mct8['REF'] = mct8.apply(lambda x: d_aa[x.AA_change[:3]], axis=1)
mct8['ALT'] = mct8.apply(lambda x: d_aa[x.AA_change[-3:]], axis=1)
mct8['Variant_short'] = mct8['REF']+mct8['ShortPosition'].astype(str)+mct8['ALT']

In [4]:
mct8[['UniProtID', 'ShortPosition', 'REF', 'ALT']]

,UniProtID,ShortPosition,REF,ALT
0,P36021,95,P,A
1,P36021,159,P,A
2,P36021,215,P,A
3,P36021,247,P,A
4,P36021,282,P,A
...,...,...,...,...
475,P36021,424,D,N
476,P36021,402,M,I
477,P36021,415,V,D
478,P36021,479,A,D


In [5]:
# Generates the input for PolyPhen2
mct8[['UniProtID', 'ShortPosition', 'REF', 'ALT']].to_csv('MCT8_PolyPhen2_input.csv', sep=' ', index=False)

In [7]:
pp_results = pd.read_csv('PolyPhen2_results.txt', sep='\t')
pp_results.shape

(480, 13)

In [8]:
pp_results

,#o_acc,o_pos,o_aa1,o_aa2,rsid,acc,pos,aa1,aa2,prediction,pph2_prob,pph2_FPR,pph2_TPR
0,P36021,95,P,A,?,P36021,95,P,A,probably damaging,0.998,0.01120,0.27300
1,P36021,159,P,A,?,P36021,159,P,A,probably damaging,0.989,0.03460,0.72300
2,P36021,215,P,A,?,P36021,215,P,A,possibly damaging,0.597,0.09190,0.87400
3,P36021,247,P,A,?,P36021,247,P,A,benign,0.226,0.12100,0.91300
4,P36021,282,P,A,?,P36021,282,P,A,probably damaging,1.000,0.00026,0.00018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,P36021,424,D,N,?,P36021,424,D,N,probably damaging,0.995,0.02770,0.68100
476,P36021,402,M,I,?,P36021,402,M,I,benign,0.025,0.18900,0.94900
477,P36021,415,V,D,?,P36021,415,V,D,probably damaging,0.994,0.02890,0.68900
478,P36021,479,A,D,?,P36021,479,A,D,probably damaging,0.998,0.01120,0.27300


---
Generates the Input for SIFT

---

In [90]:
# I directly Input the UniProt Fasta Sequence of MCT8 (short isoform)

result_html = "https://sift.bii.a-star.edu.sg/www/sift/tmp//8b2b27feb0_sequences.siftresults.matrix.html" # re-run and replace here

sift_results = pd.read_html(result_html)
sift_results = sift_results[0]
sift_results.dropna(inplace=True)
sift_results = sift_results[sift_results.pos != 'pos']
sift_results.reset_index(inplace=True, drop=True)
sift_results.index = range(87, 517)
sift_results

,pos,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
87,1G 0.23,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
88,2T 0.23,0.53,0.07,0.45,0.52,0.08,0.31,0.13,0.16,0.52,...,0.08,0.52,0.26,0.33,0.31,0.87,1.00,0.24,0.03,0.11
89,3A 0.23,0.65,0.06,0.47,0.80,0.09,0.34,0.17,0.18,1.00,...,0.10,0.41,0.25,0.56,0.69,0.58,0.48,0.27,0.03,0.12
90,4R 0.23,0.13,0.01,0.12,0.55,0.02,0.08,0.05,0.04,0.32,...,0.02,0.10,0.06,0.15,1.00,0.12,0.10,0.06,0.01,0.03
91,5G 0.54,0.36,0.04,0.10,0.07,0.02,1.00,0.02,0.02,0.07,...,0.02,0.10,0.24,0.04,0.05,0.21,0.08,0.06,0.01,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,426S 0.46,0.71,0.05,0.40,0.62,0.06,0.26,0.10,0.34,0.76,...,0.08,0.35,0.20,0.38,0.34,0.90,1.00,0.25,0.02,0.07
513,427S 0.46,1.00,0.09,0.12,0.13,0.04,0.67,0.03,0.05,0.13,...,0.04,0.12,0.22,0.09,0.09,0.80,0.20,0.15,0.01,0.04
514,428K 0.46,0.37,0.03,0.43,0.97,0.03,0.34,0.09,0.08,1.00,...,0.05,0.29,0.16,0.35,0.36,0.34,0.26,0.13,0.02,0.05
515,429D 0.46,0.11,0.01,1.00,0.89,0.01,0.14,0.05,0.02,0.13,...,0.01,0.85,0.06,0.12,0.06,0.14,0.08,0.03,0.01,0.02


In [91]:
sift_results.loc[295]

pos    209G 0.92
A           0.68
C           0.07
D           0.29
E           0.24
F           0.04
G           1.00
H           0.07
I           0.06
K           0.36
L           0.11
M           0.04
N           0.30
P           0.40
Q           0.16
R           0.17
S           0.93
T           0.25
V           0.14
W           0.02
Y           0.05
Name: 295, dtype: object

In [92]:
sift_missense = mct8[['ShortPosition', 'Variant_short', 'REF', 'ALT']]

ClinSig = []
prob = []
for i in mct8.index:
    if float(sift_results.loc[mct8.loc[i, 'ShortPosition'], mct8.loc[i, 'ALT']]) <= 0.05:
        ClinSig.append(1)
        prob.append(sift_results.loc[mct8.loc[i, 'ShortPosition'], mct8.loc[i, 'ALT']])
    else:
        ClinSig.append(0)
        prob.append(sift_results.loc[mct8.loc[i, 'ShortPosition'], mct8.loc[i, 'ALT']])

sift_missense['SIFT_pred'] = ClinSig
sift_missense['SIFT_prob'] = prob
sift_missense

/tmp/ipykernel_3031/2590438914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sift_missense['SIFT_pred'] = ClinSig
/tmp/ipykernel_3031/2590438914.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sift_missense['SIFT_prob'] = prob


,ShortPosition,Variant_short,REF,ALT,SIFT_pred,SIFT_prob
0,95,P95A,P,A,1,0.00
1,159,P159A,P,A,1,0.00
2,215,P215A,P,A,1,0.00
3,247,P247A,P,A,0,1.00
4,282,P282A,P,A,1,0.04
...,...,...,...,...,...,...
475,424,D424N,D,N,0,0.99
476,402,M402I,M,I,0,0.21
477,415,V415D,V,D,1,0.01
478,479,A479D,A,D,1,0.02


In [95]:
sift_missense.to_csv('SIFT_results.txt', sep='\t')

---
Merge PolyPhen2 and SIFT results to the variants dataset

---

In [9]:
mct8 = pd.read_csv('MCT8_IA_dataset_T4.csv', index_col=0)
mct8

,AA_change,Activity,Expression,PMExpression,LongPosition,TMD,ICL,ECL,MembraneFacing,ChannelFacing,...,FoldX_ddG_HM_IW,MISA,SASA,SCSA,SIS,MAESTRO_ddG,FoldX_ddG,dCOM,evol_indices_init,evol_indices_mean
0,Pro169Ala,95.8,NaN,NaN,169,0.0,1.0,0.0,0.0,0.0,...,1.7,0.045,38.905,0.00,0.000,0.583984,2.033333,26.782026,4.484985,4.504437
1,Pro233Ala,24.7,NaN,NaN,233,1.0,0.0,0.0,1.0,0.0,...,2.8,17.645,16.210,8.43,0.000,-0.502685,2.666667,14.001623,5.286438,5.533997
2,Pro289Ala,69.8,NaN,NaN,289,1.0,0.0,0.0,0.0,0.0,...,2.3,0.000,4.510,0.00,0.000,-0.469728,2.466667,13.171551,5.843872,5.583874
3,Pro321Ala,81.1,NaN,NaN,321,1.0,0.0,0.0,0.0,0.0,...,2.9,0.000,2.270,0.00,0.000,-0.382626,2.800000,18.156483,5.011963,5.110223
4,Pro356Ala,73.1,NaN,NaN,356,0.0,1.0,0.0,0.0,0.0,...,1.2,0.040,61.120,0.00,0.000,0.503667,1.400000,23.227317,6.271729,6.504669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Asp498Asn,4.8,83.8,106.6,498,1.0,0.0,0.0,0.0,0.0,...,0.6,0.000,31.145,32.82,26.315,0.071375,1.200000,10.968726,7.860901,7.997272
476,Met476Ile,104.5,NaN,NaN,476,1.0,0.0,0.0,1.0,0.0,...,0.6,45.490,0.000,0.00,0.000,-0.363989,1.000000,22.697429,1.725952,1.793298
477,Val489Asp,40.1,NaN,NaN,489,1.0,0.0,0.0,1.0,0.0,...,-0.2,31.845,20.375,0.00,0.000,-0.645502,0.100000,24.062603,9.147888,9.454669
478,Ala553Asp,18.1,66.5,34.4,553,1.0,0.0,0.0,0.0,0.0,...,2.7,0.000,0.000,0.00,0.000,-0.089654,4.100000,21.964107,9.257019,9.531854


In [10]:
d_aa = {'Cys': 'C', 'Asp': 'D', 'Ser': 'S', 'Gln': 'Q', 'Lys': 'K',
        'Ile': 'I', 'Pro': 'P', 'Thr': 'T', 'Phe': 'F', 'Asn': 'N',
        'Gly': 'G', 'His': 'H', 'Leu': 'L', 'Arg': 'R', 'Trp': 'W',
        'Ala': 'A', 'Val': 'V', 'Glu': 'E', 'Tyr': 'Y', 'Met': 'M',
        'Ter': 'STOP'}
aa_d = {k:v for v,k in d_aa.items()}

In [11]:
mct8['ShortPosition'] = mct8.LongPosition-74
mct8['REF'] = mct8.apply(lambda x: d_aa[x.AA_change[:3]], axis=1)
mct8['ALT'] = mct8.apply(lambda x: d_aa[x.AA_change[-3:]], axis=1)
mct8['Variant_short'] = mct8['REF']+mct8['ShortPosition'].astype(str)+mct8['ALT']

In [12]:
mct8 = mct8[['AA_change', 'Variant_short', 'Activity']]

In [14]:
pp_results = pd.read_csv('PolyPhen2_results.txt', sep='\t')
pp_results['Variant_short'] = pp_results['o_aa1'].str.strip()+pp_results[' o_pos'].astype(str).str.strip()+pp_results['o_aa2'].str.strip()
pp_results = pp_results[['Variant_short', '        prediction', ' pph2_prob']]
pp_results.rename(columns={'        prediction':'PolyPhen2_pred', ' pph2_prob':'PolyPhen2_prob'}, inplace=True)
pp_results

,Variant_short,PolyPhen2_pred,PolyPhen2_prob
0,P95A,probably damaging,0.998
1,P159A,probably damaging,0.989
2,P215A,possibly damaging,0.597
3,P247A,benign,0.226
4,P282A,probably damaging,1.000
...,...,...,...
475,D424N,probably damaging,0.995
476,M402I,benign,0.025
477,V415D,probably damaging,0.994
478,A479D,probably damaging,0.998


In [15]:
mct8 = mct8.merge(pp_results, on='Variant_short', how='left')

In [16]:
sift_results = pd.read_csv('SIFT_results.txt', sep='\t', index_col=0)
sift_results = sift_results[['Variant_short', 'SIFT_pred', 'SIFT_prob']]
sift_results

,Variant_short,SIFT_pred,SIFT_prob
0,P95A,1,0.00
1,P159A,1,0.00
2,P215A,1,0.00
3,P247A,0,1.00
4,P282A,1,0.04
...,...,...,...
475,D424N,0,0.99
476,M402I,0,0.21
477,V415D,1,0.01
478,A479D,1,0.02


In [17]:
mct8 = mct8.merge(sift_results, on='Variant_short', how='left')

In [18]:
mct8

,AA_change,Variant_short,Activity,PolyPhen2_pred,PolyPhen2_prob,SIFT_pred,SIFT_prob
0,Pro169Ala,P95A,95.8,probably damaging,0.998,1,0.00
1,Pro233Ala,P159A,24.7,probably damaging,0.989,1,0.00
2,Pro289Ala,P215A,69.8,possibly damaging,0.597,1,0.00
3,Pro321Ala,P247A,81.1,benign,0.226,0,1.00
4,Pro356Ala,P282A,73.1,probably damaging,1.000,1,0.04
...,...,...,...,...,...,...,...
475,Asp498Asn,D424N,4.8,probably damaging,0.995,0,0.99
476,Met476Ile,M402I,104.5,benign,0.025,0,0.21
477,Val489Asp,V415D,40.1,probably damaging,0.994,1,0.01
478,Ala553Asp,A479D,18.1,probably damaging,0.998,1,0.02


In [19]:
mct8.to_csv('mct8_predictors.csv')